In [2]:
from models.arcs import load_alt_model_a, load_inf_model, GenPhiloText
from utilities.loaders import load_file
from utilities.preprocessors import preprocess, map_value_to_index, init_sequences_a, init_sequences_b, decode_predictions
from utilities.visualizers import export_results

from tensorflow.keras.losses import CategoricalCrossentropy as cce_loss
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import CategoricalAccuracy, CategoricalCrossentropy as cce_metric

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow as tf

%load_ext autoreload
%autoreload 2

In [3]:
corpus = load_file('./data/notes.txt')

In [4]:
corpus[:500]

'A simple idea “What is the meaning of life?”\nI asked as I learned through the works of Camus? \nOne step down, I felt a yearning of meaning in this world.\nIn this yearning I stumbled upon eastern philosophy;\nIkigai as the Japanese philosophers called it was a considerable way for me to find meaning at that certain point in my life. \nFollowed then another idea, a leap of faith as Kierkegaard would call it, yet I had no idea this was his idea. \nCalm followed after the storm, and then I took another'

In [5]:
len(corpus)

226750

In [6]:
chars = sorted(list(set(corpus)))
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '…']

# Preprocessing corpus
* replace quotation marks like this '“'/'”' with this instead '"'
* replace single quotation marks like this '‘'/'’' with ''' instead
* replace this hyphen '–' with this hyphen '—'
* lowercase all words (for now)
* replace 3 consecutive '.' with  '…' instead

In [7]:
corpus = preprocess(corpus)
corpus[:500]

'a simple idea "what is the meaning of life?"\ni asked as i learned through the works of camus? \none step down, i felt a yearning of meaning in this world.\nin this yearning i stumbled upon eastern philosophy;\nikigai as the japanese philosophers called it was a considerable way for me to find meaning at that certain point in my life. \nfollowed then another idea, a leap of faith as kierkegaard would call it, yet i had no idea this was his idea. \ncalm followed after the storm, and then i took another'

In [8]:
len(corpus)

226961

In [9]:
chars = sorted(list(set(corpus)))
# chars = ['[UNK]'] + chars
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

# Creating mapper from a unique character to its respective index

In [10]:
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [11]:
char_to_idx

In [12]:
idx_to_char

In [13]:
n_unique = len(char_to_idx.get_vocabulary())
n_unique

57

In [14]:
n_time_steps = 100
X, Y = init_sequences_a(corpus, char_to_idx, T_x=n_time_steps)
X

<tf.Tensor: shape=(226861, 100), dtype=int64, numpy=
array([[25,  2, 43, ..., 29,  2, 43],
       [ 2, 43, 33, ...,  2, 43, 44],
       [43, 33, 37, ..., 43, 44, 29],
       ...,
       [39, 42,  2, ..., 29, 36, 33],
       [42,  2, 44, ..., 36, 33, 29],
       [ 2, 44, 42, ..., 33, 29, 30]], dtype=int64)>

In [15]:
X.shape

TensorShape([226861, 100])

In [16]:
Y

<tf.Tensor: shape=(226861,), dtype=int64, numpy=array([44, 29, 40, ..., 29, 30, 11], dtype=int64)>

# convert Y data's indeces to their one hot vector representation

In [17]:
Y = tf.one_hot(Y, depth=n_unique)
Y

<tf.Tensor: shape=(226861, 57), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [18]:
Y[3]

<tf.Tensor: shape=(57,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)>

In [19]:
# number of examples
len(X)

226861

In [20]:
len(X[-1])

100

# Instantiate generative model A with set architecture

In [21]:
model = load_alt_model_a(n_unique=n_unique, T_x=n_time_steps, emb_dim=32, n_a=128)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 32)           1824      
                                                                 
 lstm (LSTM)                 [(None, 100, 128),        82432     
                              (None, 128),                       
                              (None, 128)]                       
                                                                 
 dense (Dense)               (None, 100, 57)           7353      
                                                                 
Total params: 91609 (357.85 KB)
Trainable params: 91609 (357.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Provide loss, optimizer, and metrics for both alternative models A and B

In [22]:
opt = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
loss = cce_loss()
metrics = [CategoricalAccuracy(), cce_metric()]

# model.compile(loss=loss, optimizer=opt, metrics=metrics)

# Train alternative model A and checkpoint weights

In [23]:
# weights_path = "./weights/weights-improvement-{epoch:02d}-{categorical_accuracy:.4f}.hdf5"
# checkpoint = ModelCheckpoint(weights_path, monitor='categorical_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

In [24]:
# history = model.fit(X, Y, epochs=20, batch_size=2048, callbacks=callbacks_list)

In [25]:
# export_results(history, ['loss'], image_only=False)
# export_results(history, ['categorical_accuracy'], image_only=False)

# Preprocessing for alternative model B

In [26]:
X, Y = init_sequences_b(corpus, char_to_idx, T_x=n_time_steps)
X

<tf.Tensor: shape=(2248, 100), dtype=int64, numpy=
array([[25,  2, 43, ..., 29,  2, 43],
       [29, 40,  2, ..., 36, 39, 43],
       [40, 32, 49, ..., 25, 44,  2],
       ...,
       [39,  2, 30, ..., 32, 33, 43],
       [44, 32, 29, ...,  2, 26, 42],
       [25, 35, 43, ...,  0,  0,  0]], dtype=int64)>

In [27]:
X.shape[0]

2248

In [28]:
len(X)

2248

In [29]:
len(X[-2])

100

In [30]:
len(X[-1])

100

In [31]:
len(Y[-1])

100

In [32]:
X[-1]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([25, 35, 43,  2, 37, 49,  2, 26, 29, 36, 33, 29, 30,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int64)>

In [33]:
Y

<tf.Tensor: shape=(2248, 100), dtype=int64, numpy=
array([[ 2, 43, 33, ...,  2, 43, 44],
       [40,  2, 28, ..., 39, 43, 39],
       [32, 49, 23, ..., 44,  2, 44],
       ...,
       [ 2, 30, 33, ..., 33, 43, 56],
       [32, 29,  2, ..., 26, 42, 29],
       [35, 43,  2, ...,  0,  0,  0]], dtype=int64)>

In [34]:
Y = [tf.one_hot(y, depth=n_unique) for y in tf.reshape(Y, shape=(-1, Y.shape[0]))]

In [35]:
len(Y)

100

# Y here is now a matrix with shape $(T_y, m, n_{unique})$

In [36]:
Y

[<tf.Tensor: shape=(2248, 57), dtype=float32, numpy=
 array([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2248, 57), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2248, 57), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2248, 57), dtype=float32, 

# Load a saved model
* see arhictecure
* see if prediction will work properly on dummy data

In [37]:
# saved_model = load_model('./saved/models/test_model_b.h5')

In [38]:
# saved_model.summary()

#### Recall that our model needs 3 inputs, X, the hidden state, and the cell state. Because we are generating novel sequences using our trained model we pass in a $(1, 100)$ input where it represents the shape $(m, T_x)$, moreover our hidden and cell states remain the same in terms of their shape which is $(m, n_a)$ but only now it would be $(1, n_a)$ since we are passing only one input example to our model

In [39]:
sample_input = tf.random.uniform(shape=(1, 100), minval=0, maxval=n_unique - 1, dtype=tf.int32)

sample_h = tf.zeros(shape=(1, 64))
sample_c = tf.zeros(shape=(1, 64))

#### use the model to predict an output Y which we know will be of shape $(T_y, m, n_{unique})$ or in this case since we only inputted one example $(T_y, 1, 26)$

In [40]:
# saved_model.predict([sample_input, sample_h, sample_c])

#### List all layer names
* the goal here is to extract the Embedding, LSTM, Dense, and BatchNormalization layers used in training the model which have all been instantiated once and where we can extract and use as objects in our inference model
* Once we know the layers, we can start by training our model for training, and save the best model with the lowest loss value
* Access the saved models aforementioned layers and use it in the inference model

In [41]:
# layers = saved_model.layers
# for layer in layers:
#     print(layer.name)

In [42]:
# lstm_cell = saved_model.get_layer('lstm-cell')
# embedding_layer = saved_model.get_layer('character-lookup')
# dense_layers = []
# norm_layers = []
# for layer in layers:
#     if "dense" in layer.name:
#         dense_layers.append(saved_model.get_layer(layer.name))
#     if "norm" in layer.name:
#         norm_layers.append(saved_model.get_layer(layer.name))

In [43]:
# lstm_cell.get_weights()

In [44]:
# embedding_layer.get_weights()

In [45]:
# for dense_layer in dense_layers:
#     print(dense_layer.get_weights())

In [46]:
# for norm_layer in norm_layers:
#     print(norm_layer.get_weights())

# Load saved weights

In [47]:
saved_model = GenPhiloText(emb_dim=32, n_a=64, n_unique=57, T_x=100, dense_layers_dims=[57], lambda_=0.8, drop_prob=0.4, normalize=False)
saved_model([sample_input, sample_h, sample_c])

[<tf.Tensor: shape=(1, 57), dtype=float32, numpy=
 array([[-0.00737464, -0.00351802, -0.00758541,  0.00174319,  0.00277107,
          0.00389953,  0.00295227, -0.00182492,  0.00700726, -0.00172334,
          0.00029356, -0.00106458, -0.00541241,  0.00123529, -0.00068121,
          0.00266673,  0.00303944,  0.00872133, -0.00131038, -0.00563951,
         -0.00015233, -0.00480324, -0.00081927,  0.00428673, -0.00110526,
         -0.00079137, -0.00230247,  0.00153852,  0.00155297, -0.00126614,
         -0.0013962 ,  0.00163668,  0.00318896,  0.0011777 ,  0.00167638,
         -0.00286045, -0.0039019 ,  0.00181255,  0.00063758,  0.00151042,
          0.00611146,  0.00029504, -0.00114854,  0.00026375, -0.00401113,
         -0.00423641,  0.00305444, -0.00347038, -0.00048552, -0.00227108,
          0.00017163, -0.00294309,  0.00262733,  0.0046073 ,  0.00209214,
         -0.00039567,  0.00421368]], dtype=float32)>,
 <tf.Tensor: shape=(1, 57), dtype=float32, numpy=
 array([[-4.4708964e-03, -4.7795

In [48]:
saved_model.summary()

Model: "gen_philo_text"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 char-emb-layer (Embedding)  multiple                  1824      
                                                                 
 lstm-cell (LSTM)            multiple                  24832     
                                                                 
 dense-layer-0 (Dense)       multiple                  3705      
                                                                 
 reshape-layer (Reshape)     multiple                  0         
                                                                 
Total params: 30361 (118.60 KB)
Trainable params: 30361 (118.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
saved_model.load_weights(filepath='./saved/weights/test_gen_philo_text_25_404.3073.h5')

In [50]:
saved_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 57s 57s/step


[array([[ 0.00268138, -0.00402186,  0.01767015,  0.00080303,  0.00336233,
         -0.00387015,  0.00697783, -0.00345316, -0.00915194,  0.01162052,
          0.00438891,  0.00738984, -0.00092794,  0.00235539, -0.01025556,
          0.00263806, -0.00135051, -0.00836865, -0.01242171,  0.01108334,
          0.00434441,  0.00080882,  0.01055648, -0.01301859, -0.00070465,
          0.00351015, -0.0009351 , -0.00112354,  0.00109496, -0.01032696,
          0.00116067,  0.004819  , -0.00017076, -0.00617258, -0.00151929,
         -0.01085526,  0.00385918,  0.00266434, -0.00317072, -0.01058194,
         -0.01169897, -0.00693264,  0.01234225, -0.00248932, -0.00023434,
          0.00676006, -0.00402324, -0.00909459, -0.00376364,  0.00245353,
         -0.00078862, -0.00036936, -0.00012412,  0.00591708,  0.00414729,
          0.0051702 ,  0.00226547]], dtype=float32),
 array([[ 0.00268105, -0.00402316,  0.01767038,  0.00080572,  0.00336215,
         -0.00387181,  0.00697796, -0.00345281, -0.00915227

In [51]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

char-emb-layer
lstm-cell
dense-layer-0
reshape-layer


In [52]:
char_emb_layer = saved_model.get_layer('char-emb-layer')
char_emb_layer.get_weights()

[array([[-9.0737842e-08, -3.0973482e-07,  2.0753367e-07, ...,
          7.3704973e-07, -4.5055467e-07,  3.4894814e-08],
        [ 2.9487580e-07, -6.2942718e-10,  1.4801060e-08, ...,
          5.3042481e-07, -3.2297467e-07,  2.7761416e-08],
        [ 3.3661976e-07,  1.5733130e-07, -6.1213495e-07, ...,
          5.1900952e-07, -2.2046856e-07,  4.6141423e-08],
        ...,
        [ 1.7429321e-07,  1.0311778e-06, -2.8859250e-07, ...,
          1.3792136e-07, -1.5107750e-07, -8.1327769e-08],
        [-8.3088118e-08,  2.3703348e-07, -8.8779146e-08, ...,
          3.8307959e-07,  8.7068265e-08,  1.2832874e-07],
        [ 1.7378098e-07,  7.1667216e-08, -1.8609222e-07, ...,
          4.2168554e-07, -4.4051643e-07, -8.0382208e-08]], dtype=float32)]

In [53]:
lstm_cell = saved_model.get_layer('lstm-cell')
lstm_cell.get_weights()

[array([[-0.11918144, -0.08921611, -0.08504646, ..., -0.11078494,
         -0.12829646,  0.11241283],
        [ 0.00974015, -0.10872313,  0.12771942, ...,  0.01652044,
          0.12637797, -0.0131928 ],
        [-0.08716875,  0.07143671, -0.03143599, ...,  0.00040502,
         -0.02733815,  0.12930533],
        ...,
        [ 0.09607493, -0.14344476,  0.03661009, ..., -0.11781833,
         -0.05609845, -0.15447684],
        [ 0.09161612, -0.10706965,  0.09418292, ..., -0.13171898,
         -0.08672424, -0.05997169],
        [-0.02366584, -0.02598683,  0.07205976, ...,  0.01050098,
          0.11546022,  0.03991429]], dtype=float32),
 array([[ 0.13180849, -0.06335574,  0.022807  , ..., -0.1057215 ,
         -0.0193168 ,  0.03308445],
        [ 0.1277653 , -0.0170569 , -0.08068063, ..., -0.01837359,
         -0.04308027,  0.02766861],
        [-0.09296495, -0.0700151 , -0.03392957, ...,  0.01482407,
         -0.04170345,  0.09359735],
        ...,
        [ 0.007959  , -0.06638861, -0.0

In [54]:
dense_layers = []
norm_layers = []
for layer in layers:
    if "dense" in layer.name:
        dense_layers.append(saved_model.get_layer(layer.name))
    if "norm" in layer.name:
        norm_layers.append(saved_model.get_layer(layer.name))

In [55]:
for dense_layer in dense_layers:
    print(dense_layer.get_weights())

[array([[-9.3309274e-07, -1.3236840e-06, -1.6953194e-06, ...,
         1.3842714e-06, -5.1368502e-06,  2.4993456e-06],
       [ 1.7572141e-06, -2.8854681e-06, -2.7374339e-05, ...,
        -1.5536539e-06, -1.4193813e-06, -8.8370170e-06],
       [ 8.5380052e-06,  1.0596607e-04, -1.4875204e-05, ...,
         6.0573466e-06, -9.6267704e-06,  1.3238878e-05],
       ...,
       [ 1.0838550e-05, -6.3500545e-07,  3.6529898e-06, ...,
         9.2324299e-08, -5.9562939e-05, -1.7463933e-07],
       [-2.1821672e-05, -1.0214194e-05,  4.5075212e-05, ...,
         5.0199822e-05,  2.4682344e-05, -4.3715308e-06],
       [ 6.2264570e-07, -1.0116719e-05, -5.5610861e-05, ...,
        -7.8752946e-06, -6.7291685e-06,  3.2071886e-05]], dtype=float32), array([ 0.00268168, -0.00402042,  0.01766981,  0.0008    ,  0.00336269,
       -0.00386832,  0.00697751, -0.00345356, -0.00915158,  0.01162066,
        0.00438997,  0.00738972, -0.00092779,  0.00235699, -0.01025529,
        0.00263891, -0.00135135, -0.00836803, 

In [56]:
for norm_layer in norm_layers:
    print(norm_layer.get_weights())

In [58]:
inference_model = load_inf_model(char_emb_layer, lstm_cell, dense_layers, char_to_idx=char_to_idx, temperature=0.7)

57
64
tf.Tensor(
[[-inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]], shape=(1, 57), dtype=float32)
100
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [59]:
inference_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 char-emb-layer (Embedding)  (None, 1, 32)                1824      ['input_2[0][0]',             
                                                                     'tf.random.categorical[0][0]'
                                                                    , 'tf.random.categorical_1[0][
                                                                    0]',                          
                                                                     'tf.random.categorical_2[0][0
                                                                    ]',                     

In [60]:
sample_input = tf.random.uniform(shape=(1, 1), minval=0, maxval=n_unique - 1, dtype=tf.int32)

sample_h = tf.random.normal(shape=(1, 64))
sample_c = tf.random.normal(shape=(1, 64))

In [61]:
sample_input

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[33]])>

In [62]:
sample_h

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[ 0.14850819,  0.6729012 ,  0.788409  ,  1.7808107 , -0.0577396 ,
        -0.28560612,  0.35802004, -1.4782249 ,  1.1858126 , -0.1146072 ,
         0.06068483,  0.50278795,  0.3833409 , -0.97714317,  0.20212926,
         0.27413028,  1.315758  ,  0.4271633 ,  0.31787676, -0.06416921,
         1.4717551 ,  0.24432038, -0.39372122,  0.44087952, -0.17155302,
        -1.0954393 ,  0.12215568, -0.18386047,  1.4351362 , -0.14658083,
         0.4368573 , -1.1743683 ,  0.12213299,  0.3125699 , -0.40009275,
        -0.01477224, -0.1499104 , -0.7032583 ,  1.4065851 , -0.899069  ,
         1.0477244 , -0.5151407 , -2.2662847 ,  1.3259908 ,  0.87246156,
        -0.6540155 , -1.0542735 ,  0.32323787, -0.79587984, -1.38351   ,
        -0.6016508 , -0.66403216,  0.7677887 , -1.5155475 , -0.82696575,
        -1.935495  ,  0.14804296,  0.51254696, -0.33192658, -0.42693886,
        -0.5871948 ,  0.5410022 ,  1.0058606 ,  0.3942923 ]],
      dtype=f

In [63]:
pred_ids = inference_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 48s 48s/step


In [64]:
pred_ids

[array([[42]], dtype=int64),
 array([[21]], dtype=int64),
 array([[55]], dtype=int64),
 array([[11]], dtype=int64),
 array([[27]], dtype=int64),
 array([[24]], dtype=int64),
 array([[7]], dtype=int64),
 array([[2]], dtype=int64),
 array([[2]], dtype=int64),
 array([[29]], dtype=int64),
 array([[56]], dtype=int64),
 array([[4]], dtype=int64),
 array([[55]], dtype=int64),
 array([[26]], dtype=int64),
 array([[33]], dtype=int64),
 array([[56]], dtype=int64),
 array([[11]], dtype=int64),
 array([[21]], dtype=int64),
 array([[28]], dtype=int64),
 array([[45]], dtype=int64),
 array([[52]], dtype=int64),
 array([[39]], dtype=int64),
 array([[40]], dtype=int64),
 array([[25]], dtype=int64),
 array([[30]], dtype=int64),
 array([[25]], dtype=int64),
 array([[34]], dtype=int64),
 array([[17]], dtype=int64),
 array([[14]], dtype=int64),
 array([[50]], dtype=int64),
 array([[13]], dtype=int64),
 array([[13]], dtype=int64),
 array([[55]], dtype=int64),
 array([[50]], dtype=int64),
 array([[53]], dty

In [65]:
decode_predictions(pred_ids, idx_to_char)

'r9—.c?(  e…"—bi….9duçopafaj41z00—zé—…&ç\'\nu…7x"1j7s0.pe´90zwgbu1?t…mhi\n\'j´j (y3s\nc\n8kesd…r,—\n?çy—jd,´'